In [ ]:
#dependencies
!pip install datasets
!pip install transformers
!pip install --upgrade transformers
!pip install transformers[sentencepiece]
!pip install transformers[torch]
!pip install einops
!pip install openai
!pip install evaluate
!pip install torch
!pip install tensorflow
!pip install scikit-learn
!pip install tf-keras
!pip install accelerate -U
!pip install sentencepiece
!pip install huggingface_hub

Load dependencies

In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets
import pandas as pd
import numpy as np
import random
import torch
from functools import partial

from datasets import load_dataset
# from google.colab import auth
import openai
import os
from openai import OpenAI

import pandas as pd
# from google.colab import drive
import zipfile
import tempfile
import shutil

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

import evaluate
import torch.nn as nn
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load dataset and create sample dataset

In [ ]:
#load dataset
def load_and_extract_dataset(zip_path):
    # Create a temporary directory
    temp_dir = '/tmp/dataset_extracted'
    
    # Ensure the directory is clean
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)
    
    # Unzip the file to the temporary directory
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    # Load the dataset from the extracted directory
    dataset = load_from_disk(temp_dir)

    return dataset

In [ ]:
zip_name = 'dataset.zip'
dataset = load_and_extract_dataset(zip_name)
dataset

In [ ]:
dataset_sampled_0 = []
dataset_sampled_1 = []
def sampleDataset(samples):
    train_dataset = dataset['train']

    dataset_sentiment_0 = train_dataset.filter(lambda x: x['label'] == 0)
    dataset_sentiment_1 = train_dataset.filter(lambda x: x['label'] == 1)

    dataset_sampled_0 = dataset_sentiment_0.shuffle().select(range(samples//2))
    dataset_sampled_1 = dataset_sentiment_1.shuffle().select(range(samples//2))

    dataset_combined = concatenate_datasets([dataset_sampled_0, dataset_sampled_1])

    sampled_dataset = dataset_combined.shuffle()

    print("Positive: ", sum(1 for example in sampled_dataset if example['label'] == 1))
    print("Negative: ", sum(1 for example in sampled_dataset if example['label'] == 0))
    print(sampled_dataset)

    update_fn = partial(update_column, positivePrompt = positivePrompt, negativePrompt = negativePrompt)
    sampled_dataset = sampled_dataset.map(update_fn, batched=False)
    
    return sampled_dataset


def update_column(example, positivePrompt, negativePrompt):
    usedPrompt = positivePrompt if example["label"] == 1 else negativePrompt
    # usedString = '\n\nPositive: ' if example ["label"] == 1 else '\n\nNegative: "' 
    
    example["text"] = usedPrompt  + example["text"] + '"'
    return example    

K-shot

In [ ]:
positivePrompt = '''
Generate a positive social media tweet on a specific topic. The positive tweet should express enthusiasm or praise. Model your generated tweets after these examples:
'''
negativePrompt = '''
Generate a negative social media tweet on a specific topic. The negative tweet should express convey criticism or disappointment. Model your generated tweets after these examples: '''

prompt = 'Generate one positive tweet and one negative tweet on a very specific topic. Ensure that each tweet is enclosed in straight quotation marks (""). The positive tweet should express enthusiasm or praise, and the negative tweet should convey criticism or disappointment. Your comments should include specific features, aspects, or things that are praised/critisized.  "'


In [ ]:
sampled_dataset = sampleDataset(1000)
sampled_dataset

In [ ]:
positivePrompt += str(dataset_sampled_1[:k])
positivePrompt += '''\nEnsure your tweet is enclosed in straight double quotation marks. Provide ONLY one tweet. Positive: "'''

negativePrompt += str(dataset_sampled_0[:k])
negativePrompt += '''\nEnsure your tweet is enclosed in straight double quotation marks. Provide ONLY one tweet. Positive: "'''

In [ ]:
# Function to load the dataset and pick K random entries
def pick_random_entries(K):
    random_entries = random.sample(list(sampled_dataset), K)

    return random_entries

k = 2

random_entries = pick_random_entries(k)

promptCreated = ""

for entry in random_entries:
  promptCreated += 'POSITIVE: "' if entry['label'] == 1 else 'NEGATIVE: "'
  promptCreated += entry["text"] + '"\n'
print(promptCreated)

In [ ]:
def deleteIrrelevent(s, prompt, isPositive):
    s = s.replace(prompt, '')

    # Find the index of the second double quotation mark
    first_quote_index = s.find('"')
    if first_quote_index == -1:
        # No double quotation mark found
        return s
    
    second_quote_index = s.find('"', first_quote_index + 1)
    if second_quote_index == -1:
        # Only one double quotation mark found
        return s

    findTweets(s, prompt)
    
    # Return the substring up to the second double quotation mark
    return s[:second_quote_index + 1]

Mistral 7b model

In [ ]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = "mistralai/Mistral-7B-Instruct-v0.3"
# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_name = "distilbert/distilbert-base-uncased"
GANtokenizer = AutoTokenizer.from_pretrained(model_name)
GANmodel = AutoModelForCausalLM.from_pretrained(model_name)
GANmodel.load_state_dict(torch.load('model_state_dict.pth'))

torch.cuda.empty_cache()

if(torch.cuda.is_available()):
  GANmodel = GANmodel.cuda()

if GANtokenizer.pad_token is None:
    GANtokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
def count_rows_with_hashtags(dataset, column_name):
    """
    Count how many rows in the Dataset contain a hashtag in the specified column.
    
    Parameters:
    dataset (Dataset): The Dataset to search through.
    column_name (str): The name of the column to search for hashtags.
    
    Returns:
    int: The count of rows containing hashtags in the specified column.
    """
    # Check if the column exists in the dataset
    if column_name not in dataset.column_names:
        raise ValueError(f"Column '{column_name}' does not exist in the dataset.")
    
    # Define a function to check if a string contains a hashtag
    def contains_hashtag(text):
        return isinstance(text, str) and '#' in text
    
    # Apply the function to the specified column and sum the results
    count = sum(contains_hashtag(row[column_name]) for row in dataset)
    
    return count

count_rows_with_hashtags(dataset["train"], "text")

In [ ]:
def promptModel(prompt, length):
    inputs = GANtokenizer(prompt, return_tensors='pt').to(device)
    attention_mask = inputs['attention_mask'].to(device)

    outputs = GANmodel.generate(
        inputs['input_ids'],
        do_sample=True,
        attention_mask=attention_mask,
        num_return_sequences=1,
        max_length=length,
        temperature = 0.9, 
        top_p = 0.9, 
        repetition_penalty=1.2,
        output_scores=True,
        return_dict_in_generate=True
    )

    # Decode the generated text
    text = GANtokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    print("Generated Text:", text)

    with torch.no_grad():
        # Forward pass to get logits
        logits = GANmodel(outputs.sequences).logits


    return text, logits

In [ ]:
# output = promptModel(negativePrompt, 66)
# x = output[0]

# print(x)

In [ ]:
def findTweets(generated_text, isPositive):
    output = generated_text.replace('“', '"').replace('”', '"')

    if(isPositive):
        positive_match = re.search(r'Positive:\s*"\s*([^"]*)\s*"', output, re.DOTALL)
        positive_tweet = positive_match.group(1).strip() if positive_match else "-1"
        return positive_tweet

    else:
        negative_match = re.search(r'Negative:\s*"\s*([^"]*)\s*"', output, re.DOTALL)
        negative_tweet = negative_match.group(1).strip() if negative_match else "-1"
        return negative_tweet

In [ ]:
# y = findTweets(x, False)
# print(y)

In [ ]:
def generateAndExtractTweets(prompt, label, length):
            
    while True:
        print("\n\nSTEP 1: INITAL PROMPT")
        text, logits = promptModel(prompt, length)
        print("\n\nSTEP 2: TWEET SEARCH")
        tweet = findTweets(text, label == 1)
        
        if tweet != "-1" and len(tweet) > 0:
            print("\n\nSTEP 3: RE PROMPTING")
            length = len(GANtokenizer.encode(tweet, add_special_tokens=True))
            text, logits = promptModel(tweet, length + 1)  
            return text, logits

        print("\n\n\nretry!!")


In [ ]:
generateAndExtractTweets(positivePrompt, 1, 100)

In [ ]:
import torch.optim as optim
from transformers import AdamW

class Discriminator(nn.Module):
  def __init__(self, vocab_size, embed_size):
    super(Discriminator, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, 128, batch_first=True)
    self.fc = nn.Linear(128, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, input_ids):
    embeds = self.embedding(input_ids)
    _, (hidden, _) = self.lstm(embeds)
    output = self.fc(hidden[-1])
    return self.sigmoid(output)
#generator and discriminator instantiation
generator = GANmodel

if(torch.cuda.is_available()):
  discriminator = Discriminator(len(GANtokenizer), 768).cuda()
else:
    discriminator = Discriminator(len(GANtokenizer), 768)

#optimizers
optimizerG = AdamW(generator.parameters(), lr=5e-5)
optimizerD = optim.Adam(discriminator.parameters(), lr=5e-5)
#loss function
criterion = nn.BCEWithLogitsLoss()
data_loader = torch.utils.data.DataLoader(sampled_dataset, batch_size=1, shuffle=True)

In [ ]:
for batch in data_loader:
    print(batch.keys())
    break

In [ ]:
def tokenizeData(texts):
    return GANtokenizer(texts, truncation=True, padding=True, return_tensors='pt')

In [ ]:
i=0
for batch in data_loader:
    print(batch) 
    i+=1
    if i > 3:
        break

In [ ]:
for batch in data_loader:
        
        # Convert the input batch into a PyTorch tensor and move to GPU
    padded_inputs = GANtokenizer(batch['text'], padding=True, return_tensors="pt", truncation=True)
    real_text = padded_inputs['input_ids'].cuda()
    print(real_text)
    break

In [ ]:
import torch
import numpy as np
import os

# For more accurate error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["PYTORCH_USE_CUDA_DSA"] = "1"

batches = 1
epochs = 1
for epoch in range(epochs):
    for batch in data_loader:
        
        # Convert the input batch into a PyTorch tensor and move to GPU
        padded_inputs = GANtokenizer(batch['text'], padding=True, return_tensors="pt", truncation=True)
        real_text = padded_inputs['input_ids'].cuda()

        # Generate fake text logits using noise input
        noise = torch.randint(0, len(GANtokenizer), (1, 16)).cuda()
        outputs = generator(noise)
        fake_logits = outputs.logits.argmax(dim=-1).detach().cuda()  # Detach to avoid backpropagation
        fake_logits_raw = outputs.logits.cuda()  # Raw logits before argmax for shape consistency
        
        # Reset gradients for discriminator
        discriminator.zero_grad()

        # Create real labels tensor and move to GPU
        real_labels = torch.ones((real_text.size(0), 1), dtype=torch.float).cuda()

        # Get discriminator's prediction on real text and calculate loss
        real_output = discriminator(real_text)
        lossD_real = criterion(real_output.view(-1, 1), real_labels)
        
        # Reshape fake logits to ensure correct shape
        fake_logits = fake_logits.view(-1, 1).cuda()

        # Get discriminator's prediction on fake text and calculate loss
        fake_output = discriminator(fake_logits)
        
        # Create fake labels tensor and move to GPU
        fake_labels = torch.zeros((fake_output.size(0), 1), dtype=torch.float).cuda()
        
        lossD_fake = criterion(fake_output, fake_labels)

        # Combine real and fake losses for the discriminator
        lossD = lossD_real + lossD_fake
        lossD.backward()
        
        torch.nn.utils.clip_grad_norm_(discriminator.parameters(), max_norm=1.0)
        
        optimizerD.step()

        # Reset gradients for generator
        generator.zero_grad()

        # Generate new fake text logits
        with torch.no_grad():
            outputs = generator(noise)
        fake_logits = outputs.logits.argmax(dim=-1).cuda()

        # Get discriminator's assessment of the newly generated fake data
        fake_output = discriminator(fake_logits.view(-1, 1).cuda())

        real_labels = torch.ones((fake_output.size(0), 1), dtype=torch.float).cuda()

        # Calculate the generator's loss

        lossG = criterion(fake_output.view(-1, 1), real_labels)
        lossG.backward()
        
        torch.nn.utils.clip_grad_norm_(generator.parameters(), max_norm=1.0)

        optimizerG.step()
        
        print(f'Epoch [{epoch}/{epochs}] Loss_D: {lossD.item():.4f} Loss_G: {lossG.item():.4f}')
        print("BATCH NUMBER " + str(batches))
        batches+=1
        del padded_inputs, real_text, noise, outputs, fake_logits, fake_logits_raw, real_labels, real_output, fake_labels, fake_output
        # gc.collect(generation=2)
        torch.cuda.empty_cache()

torch.save(generator.state_dict(), 'model_state_dict.pth')

In [ ]:
generator.eval()

In [ ]:
inputs = GANtokenizer(positivePrompt, return_tensors='pt').to(device)
attention_mask = inputs['attention_mask'].to(device)

outputs = GANmodel.generate(
    inputs['input_ids'],
    do_sample=True,
    attention_mask=attention_mask,
    num_return_sequences=1,
    max_length=100,
    temperature = 0.9, 
    top_p = 0.9, 
    repetition_penalty=1.2,
    output_scores=True,
    return_dict_in_generate=True
)

# Decode the generated text
text = GANtokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
with torch.no_grad():
        # Forward pass to get logits
        initial_logits = GANmodel(outputs.sequences).logits
print("Generated Text:", text)

print("\n\n\n")

# Modify the generated text
modified_text = findTweets(text, True)
print("Modified Text:", modified_text)

# Tokenize the modified text
modified_inputs = GANtokenizer(modified_text, return_tensors='pt').to(device)
attention_mask = modified_inputs['attention_mask'].to(device)
length = len(GANtokenizer.encode(modified_text, add_special_tokens=True)) + 1

outputs = GANmodel.generate(
    modified_inputs['input_ids'],
    do_sample=True,
    attention_mask=attention_mask,
    num_return_sequences=1,
    max_length=length,
    temperature = 0.9, 
    top_p = 0.9,  
    repetition_penalty=1.2,
    output_scores=True,
    return_dict_in_generate=True
)

print("\n\n\n")

modified_text = GANtokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

# new_logits = outputs.scores

with torch.no_grad():
        # Forward pass to get logits
        new_logits = GANmodel(outputs.sequences).logits

print("Generated Text:", modified_text)

print("Logits Shape:", new_logits)
print("Logits Shape:", initial_logits)

BERT Classification

In [90]:
#load dependencies
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd

# Initialize BERT components
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# Example training data
text = str(sampled_dataset['text'][:120]) + modified_text

print(text)

labels = np.array([1] * len(sampled_dataset['text'][:20]) + [0] * len(modified_text))
labels = np.random.permutation(labels)
labels = list(labels[0:len(text)])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please u

['\nGenerate a negative social media tweet on a specific topic. The negative tweet should express convey criticism or disappointment. Model your generated tweets after these examples: Well, bags of fun I\'m going to have burning this lot off tonight    http://yfrog.com/0umftj"', '\nGenerate a negative social media tweet on a specific topic. The negative tweet should express convey criticism or disappointment. Model your generated tweets after these examples: @bmmcauliffe it wont let me upload a picture though "', '\nGenerate a negative social media tweet on a specific topic. The negative tweet should express convey criticism or disappointment. Model your generated tweets after these examples: @Hello_Candy Heh yar. And apparently most men aren\'t creative. It\'s a sad sad world "', '\nGenerate a positive social media tweet on a specific topic. The positive tweet should express enthusiasm or praise. Model your generated tweets after these examples:\nCould use a few more days down here  h

In [91]:
df = pd.DataFrame({'text': text, 'label': labels})
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'])

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Tokenize the dataset

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

In [92]:
def format_labels(example):
    example['label'] = torch.tensor(example['label'], dtype=torch.long)
    return example

train_dataset = train_dataset.map(format_labels)
test_dataset = test_dataset.map(format_labels)

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

In [102]:
# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps = 20,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.can_return_loss = True

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [103]:
# Fine-tune the model
trainer.train()
# Save the tokenizer and model
tokenizer.save_pretrained('bert_tokenizer')
model.save_pretrained('bert_model')

Epoch,Training Loss,Validation Loss
1,No log,0.487300
2,0.473900,0.556085
3,0.433700,0.640631


BERT Evaluation

In [89]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax

training_epochs = 10
score = training_epochs
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert_tokenizer')
model = BertForSequenceClassification.from_pretrained('bert_model')

def evaluate_comment(comment):
    global score
    inputs = tokenizer(comment, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1)
        prediction = torch.argmax(probs, dim=1).item()
        print(prediction)
    if prediction == 1:
        return "Human"
    else:
        score -= 1
        return "Machine"

# Example evaluation
for i in range(training_epochs):
    new_comment = modified_text[i]
    print("(" + str(i+1) + "/" + str(training_epochs) + ")", "Evaluation:", evaluate_comment(new_comment))
print("Training accuracy:", score/training_epochs)

0
(1/10) Evaluation: Machine
0
(2/10) Evaluation: Machine
0
(3/10) Evaluation: Machine
0
(4/10) Evaluation: Machine
0
(5/10) Evaluation: Machine
0
(6/10) Evaluation: Machine
0
(7/10) Evaluation: Machine
0
(8/10) Evaluation: Machine
0
(9/10) Evaluation: Machine
0
(10/10) Evaluation: Machine
Training accuracy: 0.0
